# 1) Loading corpus and queries

In [22]:
from datasets import load_dataset

ds = load_dataset("clarin-knext/fiqa-pl", "corpus")
ds2 = load_dataset("clarin-knext/fiqa-pl", "queries")
corpus = ds['corpus'].to_pandas()
queries = ds2['queries'].to_pandas()

In [26]:
queries.head()

,_id,title,text
0,0,,Co jest uważane za wydatek służbowy w podróży ...
1,4,,Wydatki służbowe - ubezpieczenie samochodu pod...
2,5,,Rozpoczęcie nowego biznesu online
3,6,,„Dzień roboczy” i „termin płatności” rachunków
4,7,,Nowy właściciel firmy – Jak działają podatki d...


# 2) Creating a dataset of positive and negative sentence pairs

First we will load the dataset that conects queries with corpus by id, question and answer.

In [17]:
ds3 = load_dataset("clarin-knext/fiqa-pl-qrels")
ds3

README.md:   0%|          | 0.00/201 [00:00<?, ?B/s]

d:\Studia\NLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adam6\.cache\huggingface\hub\datasets--clarin-knext--fiqa-pl-qrels. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train.tsv:   0%|          | 0.00/210k [00:00<?, ?B/s]

dev.tsv:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14166 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1238 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1706 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 14166
    })
    validation: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 1238
    })
    test: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 1706
    })
})

In [24]:
validation = ds3['validation'].to_pandas()
train = ds3['train'].to_pandas()
test = ds3['test'].to_pandas()

In [25]:
validation.head()

,query-id,corpus-id,score
0,1,14255,1
1,2,308938,1
2,3,296717,1
3,3,100764,1
4,3,314352,1


let's see the question and answer

In [36]:
print("Question: ", queries[queries['_id'] == '1']['text'].iloc[0])
print("Answer: ", corpus[corpus['_id'] == '14255']['text'].iloc[0])

Question:  Zgłaszanie wydatków biznesowych dla firmy bez dochodu
Answer:  Tak, możesz ubiegać się o potrącenia biznesowe, jeśli nie uzyskujesz jeszcze żadnych dochodów. Ale najpierw powinieneś zdecydować, jaką strukturę chcesz mieć dla swojej firmy. Albo struktura firmy, albo jednoosobowy przedsiębiorca lub spółka osobowa. Struktura firmy Jeśli wybierzesz strukturę firmy (która jest droższa w przygotowaniu), będziesz ubiegać się o potrącenia, ale bez dochodu. Więc ponosiłbyś stratę i kontynuowałbyś straty, dopóki dochód z firmy nie przekroczy twoich wydatków. Straty te pozostaną więc w firmie i będą mogły zostać przeniesione na przyszłe lata dochodowe, kiedy będziesz osiągać zyski, aby zrekompensować te zyski. Więcej informacji można znaleźć w ATO – Straty podatkowe firmy. Przedsiębiorca jednoosobowy o strukturze partnerskiej Jeśli zdecydujesz się być jednoosobowym przedsiębiorcą lub spółką osobową, a Twoja firma ponosi straty, musisz sprawdzić zasady dotyczące strat niekomercyjnych, a

Loading model

In [40]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_path = "clarin-knext/herbert-base-reranker-msmarco"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
features = tokenizer(['Jakie miasto jest stolica Polski?', 'Stolicą Polski jest Warszawa.'],  padding=True, truncation=True, return_tensors="pt")
model.eval()
with torch.no_grad():
    scores = model(**features).logits
    print(scores)

config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

d:\Studia\NLP\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adam6\.cache\huggingface\hub\models--clarin-knext--herbert-base-reranker-msmarco. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

tensor([[-6.0782],
        [-6.0638]])


In [62]:
separator = tokenizer.sep_token
separator

'</s>'

Preparing dataset

In [49]:
import pandas as pd

big_df = pd.concat([validation,train,test]).reset_index(drop=True)

Positive sentences

In [87]:
list_ = []

for index, row in big_df.iterrows():

  corpus_id = row['corpus-id']
  query_id = row['query-id']

  list_.append({
    'sentence': queries[queries['_id'] == str(query_id)]['text'].iloc[0] + separator + corpus[corpus['_id'] == str(corpus_id)]['text'].iloc[0],
    'is_positive': True
  })

list_[:5]

[{'sentence': 'Zgłaszanie wydatków biznesowych dla firmy bez dochodu</s>Tak, możesz ubiegać się o potrącenia biznesowe, jeśli nie uzyskujesz jeszcze żadnych dochodów. Ale najpierw powinieneś zdecydować, jaką strukturę chcesz mieć dla swojej firmy. Albo struktura firmy, albo jednoosobowy przedsiębiorca lub spółka osobowa. Struktura firmy Jeśli wybierzesz strukturę firmy (która jest droższa w przygotowaniu), będziesz ubiegać się o potrącenia, ale bez dochodu. Więc ponosiłbyś stratę i kontynuowałbyś straty, dopóki dochód z firmy nie przekroczy twoich wydatków. Straty te pozostaną więc w firmie i będą mogły zostać przeniesione na przyszłe lata dochodowe, kiedy będziesz osiągać zyski, aby zrekompensować te zyski. Więcej informacji można znaleźć w ATO – Straty podatkowe firmy. Przedsiębiorca jednoosobowy o strukturze partnerskiej Jeśli zdecydujesz się być jednoosobowym przedsiębiorcą lub spółką osobową, a Twoja firma ponosi straty, musisz sprawdzić zasady dotyczące strat niekomercyjnych, aby

Negative sentences

In [88]:
len(list_)

17110

In [89]:
for index, row in big_df.iterrows():

  corpus_id = row['corpus-id']
  query_id = row['query-id']

  for i in range(3):
    
    index = (corpus_id + i)%len(corpus)
    if corpus.iloc[index]['_id'] != corpus_id:
      list_.append({
        'sentence': queries[queries['_id'] == str(query_id)]['text'].iloc[0] + separator + corpus.iloc[index]['text'],
        'is_positive': False
      })

In [90]:
len(list_)

68440

In [93]:
from sklearn.model_selection import train_test_split

df = pd.DataFrame(list_, columns=['sentence', 'is_positive'])

temp, test = train_test_split(df, test_size=0.2, random_state=2137)
train, eval = train_test_split(temp, test_size=0.25, random_state=2137)

In [94]:
print("Train length: ", len(train), " Test length: ", len(test), " Evaluation: ", len(eval))

Train length:  41064  Test length:  13688  Evaluation:  13688


In [95]:
train.head()

,sentence,is_positive
51985,Co to jest odwrócenie niedźwiedziego paska?</s...,False
65451,"Jak mogę ustalić, czy moja stopa zwrotu jest „...",False
49524,Czy mogę mieć kilka darmowych kart kredytowych...,False
6109,"Co stanie się z moimi akcjami, gdy broker zban...",True
62381,"Jak mogę dowiedzieć się, ile licytować za miej...",False
